In [ ]:
'''
This outlines some of the code needed to create the charts in the report
'''

import pandas as pd
import numpy as np
import datetime 
from scipy.stats import norm
import math
from matplotlib import pyplot as plt
csv_path = r"eurusd_hour.csv"
eurusd = pd.read_csv(csv_path)
eurusd['date'] = pd.to_datetime(eurusd['Date'])
#Make the neccessary transforms on the data.
eurusd.columns
eurusd['MidOpen'] = (eurusd['BidOpen'] + eurusd['AskOpen'])/2
eurusd['MidClose'] = (eurusd['BidClose'] + eurusd['AskClose'])/2
eurusd['MidClose_pct_chg'] = eurusd['MidClose'].pct_change()
eurusd["bid_offer"] = eurusd['BidClose'] + eurusd['AskClose']
daily_0800 = eurusd.loc[eurusd['Hour'] == "08:00"]
daily_0800['MidClose_pct_chg'] = daily_0800['MidClose'].pct_change()
daily_1600 = eurusd.loc[eurusd['Hour'] == "16:00"]
daily_1600['MidClose_pct_chg'] = daily_1600['MidClose'].pct_change()
(mu, sigma) = norm.fit(daily_0800['MidClose_pct_chg'].dropna(how='all'))
daily_0800['MidClose_pct_chg'] = daily_0800['MidClose_pct_chg'].replace(np.nan, 0)
daily_1600['MidClose_pct_chg'] = daily_1600['MidClose_pct_chg'].replace(np.nan, 0)
# this reverses the dataframe and provides you with forward returns
eurusd['MidClose_24h_fwd'] = eurusd['MidClose_pct_chg'].iloc[::-1].rolling(24).sum().iloc[::-1]
eurusd['MidClose_24h_fwd_sign'] = eurusd['MidClose_24h_fwd'].apply(np.sign)
eurusd['MidClose_24h_prior'] = eurusd['MidClose_pct_chg'].rolling(24).sum()
eurusd['MidClose_12h_prior'] = eurusd['MidClose_pct_chg'].rolling(12).sum()
eurusd['MidClose_24h_fwd'] = eurusd['MidClose_24h_fwd'].replace(np.nan, 0)
eurusd['MidClose_24h_fwd_sign'] = eurusd['MidClose_24h_fwd_sign'].replace(np.nan, 0)
eurusd['MidClose_24h_prior'] = eurusd['MidClose_24h_prior'].replace(np.nan, 0)
eurusd['MidClose_12h_prior'] = eurusd['MidClose_12h_prior'].replace(np.nan, 0)
rolling_sd_1600 = daily_1600['MidClose_pct_chg'].rolling(252).std()*math.sqrt(252)
rolling_sd_0800 = daily_0800['MidClose_pct_chg'].rolling(252).std()*math.sqrt(252)
fig = plt.figure(1,figsize=(10,8))
ax = fig.add_subplot(111)
ax.plot(pd.to_datetime(daily_1600['Date'].iloc[252:]),rolling_sd_1600.iloc[252:], c = "b")
ax.plot(pd.to_datetime(daily_0800['Date'].iloc[252:]),rolling_sd_0800.iloc[252:], c = "r")
ax.set_xlabel("Date")
ax.set_ylabel("Standard Deviation of Returns")
ax.legend(['16:00 Daily Volatility', '08:00 Daily Volatility'])
plt.show()
ax.get_figure().savefig("EURUSDRollingVol.png")

In [ ]:
fig = plt.figure(1,figsize=(10,8))
ax = fig.add_subplot(111)
ax.scatter(eurusd['MidClose_24h_prior'].iloc[:-1], eurusd['MidClose_12h_prior'].iloc[:-1], cmap="RdYlGn", c = eurusd['MidClose_24h_fwd_sign'].loc[1:])
ax.set_xlabel("Return Momentum Last 12 Hours")
ax.set_ylabel("Return Momentum Last 24 Hours")
plt.show()
ax.get_figure().savefig("EURUSDMomentum.png")

In [ ]:
import matplotlib.mlab as mlab
#bins = 100
mu_08 = daily_0800.loc[(daily_0800['Date'] >= "2006-02-01") & (daily_0800['Date'] <= "2008-02-01"),"MidClose_pct_chg"].mean()
sd_08 = daily_0800.loc[(daily_0800['Date'] >= "2006-02-01") & (daily_0800['Date'] <= "2008-02-01"),"MidClose_pct_chg"].std()
mu_14 = daily_0800.loc[(daily_0800['Date'] >= "2014-04-01") & (daily_0800['Date'] <= "2015-04-01"),"MidClose_pct_chg"].mean()
sd_14 = daily_0800.loc[(daily_0800['Date'] >= "2014-04-01") & (daily_0800['Date'] <= "2015-04-01"),"MidClose_pct_chg"].std()
plt.figure(1, figsize=(10, 8))
n, bins, patches = plt.hist(daily_0800.loc[(daily_0800['Date'] >= "2006-02-01") & (daily_0800['Date'] <= "2008-02-01"),"MidClose_pct_chg"], 
                            normed=True, bins=100,color = "r",alpha = 0.4)
n, bins, patches = plt.hist(daily_0800.loc[(daily_0800['Date'] >= "2014-04-01") & (daily_0800['Date'] <= "2015-04-01"),"MidClose_pct_chg"], 
                            normed=True, bins=100,color = "b",alpha= 0.3)
dist_08 = mlab.normpdf( bins, mu_08, sd_08)
dist_14 = mlab.normpdf( bins, mu_14, sd_14)
l08 = plt.plot(bins, dist_08, 'r--', linewidth=2)
l14 = plt.plot(bins, dist_14, 'b*', linewidth=2)
plt.xlabel('Daily % Changes')
plt.ylabel('Frequency')
plt.title(r'Histogram of Daily Changes in EURUSD Split By Different Market regimes')
plt.legend(['Gaussian Distribution Pre 2008',"Gaussian Distribution Post 2014" ,"Pre 2008", "Post 2014"])
plt.savefig("Regimehist.png")